## [seq] import 

In [1]:
import json
import yaml
import sys
import time
import IPython.display as ipd
import pprint
from pathlib import Path
from tqdm import tqdm

import numpy as np
import torch
import torchaudio
from librosa.filters import mel as librosa_mel_fn
#import matplotlib
#matplotlib.use("Agg")
import matplotlib.pyplot as plt
from scipy.io.wavfile import write


import toybox

KeyboardInterrupt: 

## [seq] check configuration

In [2]:
config = toybox.load_yaml_and_expand_var('configs/config_exp_PBL.yaml')

In [3]:
model_name = config['model_name']
runtime_name = config['runtime_name']
print(model_name)
print(runtime_name)

gradseptts
infer4PBL


## [seq] device setting

In [2]:
import os

print(f"all cpu at using device: {os.cpu_count()}")
print(f"Number of available CPU: {len(os.sched_getaffinity(0))}") # Number of available CPUs can also be obtained. ,use systemcall at linux.

all cpu at using device: 52
Number of available CPU: 4


In [3]:
DEVICE = 'cpu' # 'cuda' or 'cpu'
DEVICE_UTMOS = 'cuda'
if torch.cuda.is_available():
    print('use cuda')
else:
    os._exit(os.EX_OK)
    print('use cpu')

device = torch.device(DEVICE_UTMOS)
print(f'device: {device}')

use cuda
device: cuda


In [4]:
# seed val from https://github.com/sarulab-speech/UTMOS22/blob/master/strong/configs/train/default.yaml
utmos_seed = 1234
toybox.set_seed(utmos_seed)

## [seq] load model

In [8]:
predictor_utmos = torch.hub.load("tarepan/SpeechMOS:v1.2.0", "utmos22_strong", trust_repo=True)

Using cache found in /work/sora-sa/.cache/torch/hub/tarepan_SpeechMOS_v1.2.0
/work/sora-sa/aoi/.venv/lib/python3.10/site-packages/torch/nn/utils/weight_norm.py:28: UserWarning: torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.
  warnings.warn("torch.nn.utils.weight_norm is deprecated in favor of torch.nn.utils.parametrizations.weight_norm.")


## [seq] setting path


In [9]:
test_ds_path = Path(config['test_datalist_path'])

if model_name == 'groundtruth':
    #/result4eval/infer4PBL/groundtruth/cuda/
    IWAV_DIR_PATH = Path('./result4eval/infer4PBL/groundtruth/cuda/wav/')
    RESULT_DIR_PATH = Path(f'./result4eval/{runtime_name}/{model_name}')
else:
    IWAV_DIR_PATH = Path(f'./result4eval/{runtime_name}/{model_name}/{DEVICE}/wav')
    RESULT_DIR_PATH = Path(f'./result4eval/{runtime_name}/{model_name}/{DEVICE}')
    
RESULT_JSON_DIR_PATH = RESULT_DIR_PATH / 'eval4PBL.json'

In [10]:
print('-------------------------------------------')
if test_ds_path.exists():
    print(f'Exists {str(test_ds_path)}')
    with open(test_ds_path) as j:
        test_ds_list = json.load(j)
    print(f'loaded {test_ds_path}')
else:
    print(f'No exist {test_ds_path}')

print('-------------------------------------------')
if IWAV_DIR_PATH.exists():
    print(f'Exists {IWAV_DIR_PATH}')
else:
    print(f'No exist {IWAV_DIR_PATH}')

print('-------------------------------------------')
if RESULT_DIR_PATH.exists():
    print(f'Exists {RESULT_DIR_PATH}')
else:
    #RESULT_DIR_PATH.mkdir(parents=True)
    print(f'No exist {RESULT_DIR_PATH}')

print('-------------------------------------------')
if RESULT_JSON_DIR_PATH.exists():
    print(f'Exists {RESULT_JSON_DIR_PATH}')
else:
    #RESULT_DIR_PATH.mkdir(parents=True)
    print(f'No exist {RESULT_JSON_DIR_PATH}')


-------------------------------------------
Exists configs/test_dataset.json
loaded configs/test_dataset.json
-------------------------------------------
Exists result4eval/infer4PBL/gradseptts/cpu/wav
-------------------------------------------
Exists result4eval/infer4PBL/gradseptts/cpu
-------------------------------------------
No exist result4eval/infer4PBL/gradseptts/cpu/eval4PBL.json


## [seq] test load wav

In [11]:
if model_name == 'groundtruth':
    ljwav_path = Path('./data/ljspeech/LJSpeech-1.1/wavs') / f"{test_ds_list[0]['name']}.wav"
    iwav_path = IWAV_DIR_PATH / f"{test_ds_list[0]['name']}.wav"
    print(ljwav_path)
    ljwav, ljsamplerate = torchaudio.load(ljwav_path)
    ipd.display(ipd.Audio(ljwav, rate=ljsamplerate))

In [12]:
iwav_path = IWAV_DIR_PATH / f'{test_ds_list[0]["name"]}.wav'
print(iwav_path)
wav, samplerate = torchaudio.load(iwav_path)
ipd.display(ipd.Audio(wav, rate=samplerate))

result4eval/infer4PBL/gradseptts/cpu/wav/LJ045-0049.wav


In [13]:
print(wav.max())

tensor(0.5814)


## [seq] test eval

In [14]:
# wav is [1, time]
score = predictor_utmos(wav, samplerate)

In [15]:
print(score)
print(type(score))
print(score.item())
print(type(score.item()))

tensor([4.3160], grad_fn=<AddBackward0>)
<class 'torch.Tensor'>
4.315978050231934
<class 'float'>


## [seq] save LJ eval_json

In [16]:
infer_data_num: int = 100 #len(test_ds_list) is 200
print(infer_data_num)

100


In [17]:
# [seq] eval ljwav
if model_name == 'groundtruth':
    score4lj_utmos_list = []
    for i in tqdm(range(infer_data_num)):
        ljfilename = test_ds_list[i]['name']
        ljwav_path = Path('./data/ljspeech/LJSpeech-1.1/wavs') / f"{ljfilename}.wav"
        print(ljwav_path)
        ljwav, ljsamplerate = torchaudio.load(ljwav_path)
        score4lj_utmos = predictor_utmos(ljwav, ljsamplerate)
        score_float = score4lj_utmos.item()
        evallj_dict = {'name': ljfilename, 'path': str(ljwav_path), 'utmos': score_float}
        score4lj_utmos_list.append(evallj_dict)
        

In [18]:
len(score4lj_utmos_list)

NameError: name 'score4lj_utmos_list' is not defined

In [19]:
if model_name == 'groundtruth':
    RESULT_ljJSON_PATH = RESULT_DIR_PATH / 'evalljPBL.json'
    if RESULT_ljJSON_PATH.exists() == False:
        with open(RESULT_ljJSON_PATH, 'w') as f:
            for entry in score4lj_utmos_list:
                f.write(json.dumps(entry) + '\n')
        print(f'Make {RESULT_ljJSON_PATH}')
    else:
        print(f'Already Exists {RESULT_ljJSON_PATH}')

# [seq] eval wav

In [20]:
infer_data_num:int = 100 # len(test_ds_list) is 200
print(infer_data_num)
print(IWAV_DIR_PATH)
print(RESULT_DIR_PATH)

100
result4eval/infer4PBL/gradseptts/cpu/wav
result4eval/infer4PBL/gradseptts/cpu


In [21]:
score_utmos_list = []
for i in tqdm(range(infer_data_num)):
    filename = test_ds_list[i]['name']
    iwav_path = IWAV_DIR_PATH / f"{filename}.wav"
    print(iwav_path)
    wav, samplerate = torchaudio.load(iwav_path)
    score_utmos = predictor_utmos(wav, samplerate)
    score_float = score_utmos.item()
    eval_dict = {'name': filename, 'path': str(iwav_path), 'utmos': score_float}
    score_utmos_list.append(eval_dict)

  0%|                                                                                                     | 0/100 [00:00<?, ?it/s]

result4eval/infer4PBL/gradseptts/cpu/wav/LJ045-0049.wav


  2%|█▊                                                                                           | 2/100 [00:00<00:26,  3.66it/s]

result4eval/infer4PBL/gradseptts/cpu/wav/LJ017-0027.wav
result4eval/infer4PBL/gradseptts/cpu/wav/LJ023-0031.wav


  3%|██▊                                                                                          | 3/100 [00:00<00:19,  4.98it/s]

...

 98%|██████████████████████████████████████████████████████████████████████████████████████████▏ | 98/100 [00:28<00:00,  3.10it/s]

result4eval/infer4PBL/gradseptts/cpu/wav/LJ018-0059.wav


 99%|███████████████████████████████████████████████████████████████████████████████████████████ | 99/100 [00:28<00:00,  2.99it/s]

result4eval/infer4PBL/gradseptts/cpu/wav/LJ029-0081.wav


100%|███████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:29<00:00,  3.42it/s]


In [22]:
len(score_utmos_list)

100

In [23]:
RESULT_JSON_PATH = RESULT_DIR_PATH / 'eval4PBL.json'
if RESULT_JSON_PATH.exists() == False:
    with open(RESULT_JSON_PATH, 'w') as f:
        for entry in score_utmos_list:
            f.write(json.dumps(entry) + '\n')
    print(f'Make {RESULT_JSON_PATH}')
else:
    print(f'Already Exists {RESULT_JSON_PATH}')

Make result4eval/infer4PBL/gradseptts/cpu/eval4PBL.json


## [seq] show result

In [11]:
#eval_target = 'gradseptts_cpu'
eval_target = 'lj_via_hifigan'

eval_info = {
    'lj': 'result4eval/infer4PBL/groundtruth/evalljPBL.json',
    'lj_via_hifigan': 'result4eval/infer4PBL/groundtruth/eval4PBL.json',
    'gradtts_cuda': 'result4eval/infer4PBL/gradtts/cuda/eval4PBL.json',
    'gradtts_cpu': 'result4eval/infer4PBL/gradtts/cpu/eval4PBL.json',
    'gradseptts_cuda': 'result4eval/infer4PBL/gradseptts/cuda/eval4PBL.json',
    'gradseptts_cpu': 'result4eval/infer4PBL/gradseptts/cpu/eval4PBL.json',
    'gradtfktts_cuda': 'result4eval/infer4PBL/gradtfktts/cuda/eval4PBL.json',
    'gradtfktts_cpu': 'result4eval/infer4PBL/gradtfktts/cpu/eval4PBL.json',
}

# gradtts:
# gradtfktts:
eval_jsonl_path = Path(eval_info[eval_target])
if eval_jsonl_path.exists() == True:
    print(f'Exist {eval_jsonl_path}')
    import json
    with open(eval_jsonl_path) as f:
        eval_jsonl_list = [json.loads(l) for l in f]
else:
    print(f'No Exists {eval_jsonl_path}')

Exist result4eval/infer4PBL/groundtruth/eval4PBL.json


In [12]:
# culc for utmos
utmos_list = [eval_jsonl_list[n]['utmos'] for n in range(len(eval_jsonl_list))]
utmos_nparr = np.array(utmos_list)

In [13]:
utmos_mean = np.mean(utmos_nparr)
utmos_var = np.var(utmos_nparr)
utmos_std = np.std(utmos_nparr)
print(f'utmos mean: {utmos_mean}')
print(f'utmos var: {utmos_var}')
print(f'utmos std: {utmos_std}')

utmos mean: 4.019014093875885
utmos var: 0.09124055665879827
utmos std: 0.30206051820586927
